In [200]:
import numpy as np
from astropy import units as u
import galpy
import astropy.io.ascii as ascii
from galpy.potential import DoubleExponentialDiskPotential, AnySphericalPotential
from galpy.potential import HernquistPotential, DehnenSphericalPotential, NFWPotential,PlummerPotential, RazorThinExponentialDiskPotential
from galpy.potential.EllipsoidalPotential import EllipsoidalPotential
from galpy.util import conversion
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import zim
#import astropy.wcs as wcs

import mysubs
#import readsnap as giz
from scipy.signal import find_peaks
import gc
import seaborn as sns
sns.set_theme(style="white", context="paper", font_scale=1.25, palette="tab10")

%matplotlib inline

In [201]:
'''# * Normalization of the distance scale and the velocity scale in galpy
ro, vo = 8.122*u.kpc, 229.*u.km/u.s

rhoo= conversion.dens_in_msolpc3(vo=vo,ro=ro)
sigo= conversion.surfdens_in_msolpc2(vo=vo,ro=ro)'''
#ro, vo= 8.5 , 221.4
#ro, vo= 8.122 , 229.
#ro, vo= 8.0, 230.
ro, vo= 8.34, 239.89
massol= conversion.mass_in_msol(vo,ro)


# M_s=1.98841    #10^30 kg
# k_pc=3.08568   #10^19 m

# rhoo= conversion.dens_in_msolpc3(vo=vo,ro=ro)          # 0.15774525078120438
# sigo= conversion.surfdens_in_msolpc2(vo=vo,ro=ro)      #1340.8346316402374

In [202]:
snapshot_data = pd.DataFrame({
    "snapshot": [580,680,780,1200],
    'snapname': ['$T_{0} - 1$ Gyr', '$T_{0}$ Gyr', '$T_{0} + 1$ Gyr', '$T_{0} + 5$ Gyr'],
    'bulge_mass':[4.5e10, 4.5e10, 4.4e10, 4.6e10],
    'disk_mass': [6.6e10, 6.6e10, 6.8e10, 6.8e10],
    'hr': [5.2,5.2,4.9,5.2],
    'xcorr': [2.393-0.4499-0.02801-0.00228-0.0002228, 2.393-0.003802+8.013e-05+1.873e-05+9.918e-08+2.277e-08, 2.393+0.4431+0.03084+0.003001+0.00029+3.756e-05, 4.972+0.008459+0.0006881+5.681e-05],
    'ycorr': [-0.2932+0.04791+0.009522+0.001403+0.0002025, -0.2932-0.00852-0.001075-0.0001222-2.17e-05-2.299e-06, -0.2932-0.03999-0.003807-0.0005602-7.006e-05-9.693e-06, -1.125-0.01305-0.001195-0.0001231],
    'zcorr': [0.5029-0.09872+0.0115+ 0.001908+0.0002156, 0.5029-0.008322-0.0003245-2.815e-05-4.307e-06-3.032e-07, 0.5029+0.05896-0.01248-0.001803-0.0001969-2.698e-05, 1.356-0.002133-0.0003273-3.388e-05],
    'PA' : [87, 89.5, 86.8, 86.5],
    'INC': [127, 125.85, 124.8, 125.5]
})
dm_data = [2.16445826e+01, 5.92639324e+11, 1.41084051e+00] #parameters for DM profile [0] = scale radius, [1] = mass, [2] = exponent alpha

In [203]:
def Bulge_potential(M_bulge):
    Re = 1.4
    b=Re/2**(1/3)
    pot_bulge_m31_m371=PlummerPotential(amp=M_bulge*u.Msun,b=1.4/(2**(1/3.))*u.kpc,ro=ro*u.kpc, vo=vo*u.km/u.s,normalize=False)
    # bulge_potential = galpy.potential.evaluatePotentials(pot_bulge_m31_m371, R*u.kpc, z0.values*u.kpc)
    # bulge_mass_equi = galpy.potential.mass(pot_bulge_m31_m371, f*u.kpc)
    return pot_bulge_m31_m371

def Disk_potential(M_disk, hr):
    hr = hr*u.kpc
    pot_disk_m31_m371= RazorThinExponentialDiskPotential(amp=M_disk*u.Msun/(2.*np.pi*hr**2), hr = hr,ro=ro*u.kpc, vo=vo*u.km/u.s, normalize=False)
    # disk_potential = np.array([galpy.potential.evaluatePotentials(pot_disk_m31_m371, R_i*u.kpc, z0_i*u.kpc) for R_i, z0_i in zip(R, z0)])
    # disk_mass_equi = galpy.potential.mass(pot_disk_m31_m371, f*u.kpc)
    return pot_disk_m31_m371

def DM_halo_potential():
    a, M_dm_halo, alpha = dm_data[0], dm_data[1], dm_data[2]
    pot_halo_m31_m371=DehnenSphericalPotential(amp=M_dm_halo*u.Msun,a=a*u.kpc,alpha=alpha,normalize=False,ro=ro*u.kpc, vo=vo*u.km/u.s)
    # halo_potential = galpy.potential.evaluatePotentials(pot_halo_m31_m371, R*u.kpc, z0.values*u.kpc)
    # dm_mass_equi = galpy.potential.mass(pot_halo_m31_m371, f*u.kpc)
    return pot_halo_m31_m371
        

In [204]:
def apply_translation(x,y,z,i):
    x = (x - snapshot_data['xcorr'][i])*40
    y = (y - snapshot_data['ycorr'][i])*40
    z= (z - snapshot_data['zcorr'][i])*40
    return x,y,z

In [205]:
def correct_for_velocity(x,y,z,vx,vy,vz):
    r = np.sqrt(x**2 + y**2 + z**2)
    idx = r<30
    vx_temp = vx[idx]
    vy_temp = vy[idx]
    vz_temp = vz[idx]
    
    vx = (vx - np.mean(vx_temp))*153.79
    vy = (vy - np.mean(vy_temp))*153.79
    vz = (vz - np.mean(vz_temp))*153.79

    return vx,vy,vz

In [206]:
def apply_rotation(PA,inc,i):
    
    x_tcorr,y_tcorr,z_tcorr = apply_translation(x,y,z,i)
    vx_corr,vy_corr,vz_corr = correct_for_velocity(x,y,z,vx,vy,vz)

    x1=x_tcorr*np.cos(PA*3.14159/180)-y_tcorr*np.sin(PA*3.14159/180)
    y1=x_tcorr*np.sin(PA*3.14159/180)+y_tcorr*np.cos(PA*3.14159/180)
    z1=z_tcorr

    vx1=vx_corr*np.cos(PA*3.14159/180)-vy_corr*np.sin(PA*3.14159/180)
    vy1=vx_corr*np.sin(PA*3.14159/180)+vy_corr*np.cos(PA*3.14159/180)
    vz1 = vz_corr

    x2 = x1
    y2=y1*np.cos(inc*3.14159/180)-z1*np.sin(inc*3.14159/180)
    z2=y1*np.sin(inc*3.14159/180)+z1*np.cos(inc*3.14159/180)

    vx2 = vx1
    vy2=vy1*np.cos(inc*3.14159/180)-vz1*np.sin(inc*3.14159/180)
    vz2=vy1*np.sin(inc*3.14159/180)+vz1*np.cos(inc*3.14159/180)

    axis_change=90
    x3 = x2
    y3=y2*np.cos(axis_change*3.14159/180)-z2*np.sin(axis_change*3.14159/180)
    z3=y2*np.sin(axis_change*3.14159/180)+z2*np.cos(axis_change*3.14159/180)

    vx3 = vx2
    vy3=vy2*np.cos(axis_change*3.14159/180)-vz2*np.sin(axis_change*3.14159/180)
    vz3=vy2*np.sin(axis_change*3.14159/180)+vz2*np.cos(axis_change*3.14159/180)

    # plt.scatter(x3,y3,s=0.1)
    # plt.xlim(-50,50)
    # plt.ylim(-50,50)
    # plt.show()

    return x3,y3,z3,vx3,vy3,vz3

In [207]:
def Angular_momentum_and_KE(PA, inc, i):
        x,y,z,vx,vy,vz = apply_rotation(PA, inc, i)
        jx = y*vz - z*vy
        jy = z*vx - x*vz
        jz = x*vy - y*vx
        
        R = np.sqrt(x**2 + y**2)
        j = np.sqrt(jx**2 + jy**2 + jz**2)
        KE = 0.5*(vx**2 + vy**2 + vz**2)
        return R, z, j, KE

In [208]:
def equilibrium(bulge_model, disk_model, halo_model, f):
    G = 4.300917270038e-06
    bulge_mass = galpy.potential.mass(bulge_model, f*u.kpc)
    disk_mass = galpy.potential.mass(disk_model, f*u.kpc)
    dm_mass = galpy.potential.mass(halo_model, f*u.kpc)
    v12 = G*(bulge_mass+disk_mass+dm_mass)/f
    
    bp = galpy.potential.evaluatePotentials(bulge_model, f*u.kpc, z=0)
    dp = np.array([galpy.potential.evaluatePotentials(disk_model, R_i*u.kpc, z=0) for R_i in f])
    dmp = galpy.potential.evaluatePotentials(halo_model, f*u.kpc, z=0) 
    potential = bp+dp+dmp #-1* #G*(M_bulge+M_bulge+M_dm)*u.Msun/r

    total_energy = potential+0.5*v12
    tot_ang_momentum = f*np.sqrt(v12)

    return tot_ang_momentum, total_energy
        
    


In [209]:
def particles_selection_35kpc(R, j, j_equi,total_energy, total_energy_equi,i):
    shifted_equilibrium = total_energy_equi + 2500
    idx = R>35
    j_filt = j[idx]
    te_filt = total_energy[idx]
    lower_bound = np.interp(j_filt, j_equi, total_energy_equi)
    upper_bound = np.interp(j_filt, j_equi, shifted_equilibrium)

    in_between_mask = (te_filt > lower_bound) & (te_filt < upper_bound)
    upper_mask = (te_filt> upper_bound)

    # particles_in_between = te_filt[in_between_mask]
    # particles_upper = te_filt[upper_mask]

    x,y,z,vx,vy,vz = apply_rotation(snapshot_data['PA'][i], snapshot_data['INC'][i], i)
    x_between, y_between, z_between = x[idx][in_between_mask], y[idx][in_between_mask], z[idx][in_between_mask]
    x_upper, y_upper, z_upper = x[idx][upper_mask], y[idx][upper_mask], z[idx][upper_mask]
    
    xlim = 500
    ylim = 500

    #plt.figure(figsize=(6,6))
    # plt.scatter(x_between, y_between, s=1)
    # plt.xlabel('x [kpc]')
    # plt.ylabel('y [kpc]')
    # plt.title('Particles in between')
    # plt.xlim(-xlim,xlim)
    # plt.ylim(-ylim,ylim)
    # plt.axis('scaled')
    # plt.show()
    # plt.scatter(x_upper, y_upper, s=0.1)
    # plt.show()
    
    return #particles_in_between, particles_upper

    
    

In [210]:
s = snapshot_data['snapshot']
for i in range(len(snapshot_data)):
    file = f'Model371/moredata/snapshot_{s[i]}.hdf5_gas.dat'
    df = pd.read_csv(file, delimiter='\s+')
    df = df[df['9-Temperature[K]'] < 20000]
    print(s[i])
    x, y, z = df['2-X'], df['3-Y'], df['4-Z']
    vx, vy, vz = df['5-Vx'], df['6-Vy'], df['7-Vz']
    

    print(snapshot_data['PA'][i], snapshot_data['INC'][i])
    R, z0, j, KE = Angular_momentum_and_KE(snapshot_data['PA'][i], snapshot_data['INC'][i], i)
    M_bulge, M_disk= snapshot_data['bulge_mass'][i], snapshot_data['disk_mass'][i]
    hr = snapshot_data['hr'][i]

    f = np.linspace(0.1,3000,10000)

    bulge_model = Bulge_potential(M_bulge)
    bulge_potential = np.array([galpy.potential.evaluatePotentials(bulge_model, R_i*u.kpc, z0_i*u.kpc) for R_i, z0_i in zip(R, z0)])

    disk_model = Disk_potential(M_disk, hr)
    disk_potential = np.array([galpy.potential.evaluatePotentials(disk_model, R_i*u.kpc, z0_i*u.kpc) for R_i, z0_i in zip(R, z0)])

    dm_model = DM_halo_potential()
    dm_potential = np.array([galpy.potential.evaluatePotentials(dm_model, R_i*u.kpc, z0_i*u.kpc) for R_i, z0_i in zip(R, z0)])
    
    total_potential = bulge_potential + disk_potential + dm_potential
    total_energy = KE + total_potential

    

    j_equi, total_energy_equi = equilibrium(bulge_model, disk_model, dm_model, f)

    radial_zone1 = R<25
    radial_zone2 = (R>25) & (R<35)
    radial_zone3 = R>35

    
    # plt.figure(figsize=(6,6))
    # plt.scatter(j[radial_zone1], total_energy[radial_zone1], s=0.1, label='R<25')
    # plt.scatter(j[radial_zone2], total_energy[radial_zone2], s=0.1, label='25<R<35')
    # plt.scatter(j[radial_zone3], total_energy[radial_zone3], s=0.1, label='R>35')
    # plt.plot(j_equi, total_energy_equi, label='Equilibrium', c='k')
    # plt.xlabel('J$(kms^{-1}kpc)$')
    # plt.ylabel('E $(km/s)^{2}$')
    # plt.xlim(0,3e4)
    # plt.ylim(-8e4,4e4)
    # plt.title(snapshot_data['snapname'][i])
    # plt.legend(scatterpoints=1, markerscale=6)
    # plt.tight_layout()
    # plt.show()

    
    particles_selection_35kpc(R, j, j_equi,total_energy, total_energy_equi,i)
   

<>:4: SyntaxWarning: invalid escape sequence '\s'

<>:4: SyntaxWarning: invalid escape sequence '\s'

/tmp/ipykernel_211509/1484028159.py:4: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file, delimiter='\s+')

580
87.0 127.0
680
89.5 125.85
780
86.8 124.8
1200
86.5 125.5
